<a href="https://colab.research.google.com/github/BhaskarFysik/ALGO2/blob/main/bh_rsi_oscillator_and_smacross.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install matplotlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install numpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install pandas_ta

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 115 kB 10.5 MB/s 
  Created wheel for pandas-ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218923 sha256=219c1ab8d9b8c026ffa4e06ad97d27c022ece7e1d761dd5bac4ea9f04595f6b1
  Stored in directory: /root/.cache/pip/wheels/0b/81/f0/cca85757840e4616a2c6b9fe12569d97d324c27cac60724c58
Successfully built pandas-ta


In [ ]:
pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 1.3 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


In [ ]:
pip install backtesting

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 175 kB 14.2 MB/s 
  Created wheel for backtesting: filename=Backtesting-0.3.3-py3-none-any.whl size=173832 sha256=d5ede9d897c8ba14b7c503f1a0ccb8fcc791bd7ac2d8069828b6b53757acfec0
  Stored in directory: /root/.cache/pip/wheels/48/1d/ea/65dacebc37da7655d8a1fd0f315ac39d102e31d6545237a1c6
Successfully built backtesting


In [ ]:
from backtesting import Strategy
from backtesting.lib import crossover
from backtesting import Backtest
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import pandas_ta as ta
#from backtesting.test import GOOG

symbol='HDFC'
df = yf.Ticker(f"{symbol}.NS").history(period="5y", interval="1d")
#df=pd.DataFrame(GOOG)
def RSI(values,n):
     return ta.rsi(close=df["Close"], length=n)
def SMA(values,n):
     return pd.Series(values).rolling(n).mean()



class RsiOscillator(Strategy):
    upper_bound=70
    lower_bound=30
    n1=20
    n2=50
    def init(self):
        self.rsi=self.I(RSI,self.data.Close,14)
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)
    def next(self):

        if crossover(self.rsi,self.upper_bound) and self.sma2>self.sma1:
            self.position.close()
        elif crossover(self.lower_bound,self.rsi) and self.sma2<self.sma1:
            self.buy()

bt=Backtest(df,RsiOscillator, cash=10000)
stats=bt.optimize(
    upper_bound=range(10,85,5),
    lower_bound=range(10,85,5),
    n1=range(5, 30, 5),
    n2=range(10, 70, 5),
    maximize='Equity Final [$]',
    constraint=lambda param: param.upper_bound>param.lower_bound and param.n1 < param.n2)
#stats=bt.run()
print(stats)
bt.plot()

ModuleNotFoundError: ignored